# Análisis Exploratorio de Datos - HR Analytics

Este notebook contiene el análisis exploratorio de los datos de recursos humanos, enfocándonos en encontrar patrones y variables correlacionadas con la variable target 'attrition'.

In [3]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configuración de visualización
plt.style.use('seaborn')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Carga de datos
df = pd.read_csv('../data/clean/encoded_data.csv')
print(f'Dimensiones del dataset: {df.shape}')
df.head()

## 1. Análisis de la Variable Target (Attrition)

Primero analizaremos la distribución de nuestra variable objetivo.

In [ ]:
# Distribución de Attrition
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Attrition')
plt.title('Distribución de Attrition')
plt.show()

# Porcentajes
attrition_pct = df['Attrition'].value_counts(normalize=True) * 100
print('\nPorcentaje de Attrition:')
print(attrition_pct)

## 2. Análisis de Variables Numéricas

In [ ]:
# Seleccionar variables numéricas
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Calcular correlaciones con Attrition
correlations = df[numeric_cols].corrwith(df['Attrition'])
correlations = correlations.sort_values(ascending=False)

# Visualizar correlaciones
plt.figure(figsize=(12, 8))
correlations.plot(kind='bar')
plt.title('Correlación de Variables Numéricas con Attrition')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print('\nTop 5 correlaciones positivas:')
print(correlations.head())
print('\nTop 5 correlaciones negativas:')
print(correlations.tail())

## 3. Análisis de Variables Categóricas

In [ ]:
# Función para crear gráficos de barras por categoría
def plot_categorical_analysis(df, column):
    plt.figure(figsize=(12, 6))
    
    # Calcular porcentajes de Attrition por categoría
    attrition_by_cat = df.groupby(column)['Attrition'].mean() * 100
    
    # Crear gráfico
    attrition_by_cat.plot(kind='bar')
    plt.title(f'Porcentaje de Attrition por {column}')
    plt.ylabel('Porcentaje de Attrition')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    print(f'\nPorcentaje de Attrition por {column}:')
    print(attrition_by_cat.sort_values(ascending=False))

In [ ]:
# Analizar variables categóricas principales
categorical_cols = ['Department', 'JobRole', 'MaritalStatus', 'Gender', 'BusinessTravel']

for col in categorical_cols:
    plot_categorical_analysis(df, col)

## 4. Análisis de Satisfacción y Feedback

In [ ]:
# Análisis de variables de satisfacción
satisfaction_cols = ['average_employee_satisfaction', 'average_manager_feedback']

for col in satisfaction_cols:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='Attrition', y=col)
    plt.title(f'{col} vs Attrition')
    plt.show()
    
    # T-test para verificar diferencias significativas
    t_stat, p_val = stats.ttest_ind(
        df[df['Attrition'] == 0][col],
        df[df['Attrition'] == 1][col]
    )
    print(f'\nT-test para {col}:')
    print(f'T-statistic: {t_stat:.4f}')
    print(f'P-value: {p_val:.4f}')

## 5. Análisis de Variables de Tiempo y Trabajo

In [ ]:
# Análisis de variables relacionadas con tiempo y trabajo
time_cols = ['YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, col in enumerate(time_cols):
    sns.boxplot(data=df, x='Attrition', y=col, ax=axes[idx])
    axes[idx].set_title(f'{col} vs Attrition')

plt.tight_layout()
plt.show()